In [ ]:
import torchvision
import torch
from models_scripts import i3_res50, i3_res50_nl
from models_scripts import I3Res50
from model_resnet18 import I3Res18, freeze_bn
import torch
import torch.optim as optim
from torchvision import datasets, transforms
from KD_Lib.KD import VanillaKD
from KD_Lib.KD import DML
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from copy import deepcopy
import torchvision.models as models
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import random
%matplotlib inline
from models_scripts import model
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
import pytorchvideo
import json
from dataset_scripts import CTDataset, CTDatasetTestSimple
from torch.utils.data import DataLoader
import json
import urllib
from pytorchvideo.data.encoded_video import EncodedVideo
from utilities_scripts import SAM, LR_Scheduler, get_criterion, LoadingBar, Log, initialize, RandAugment

from torchvision.transforms import Compose, Lambda

from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample
)

In [ ]:
num_workers = 1
fold_train_path = "./train_folds.json"
fold_test_path = "./test_folds.json"
fold_id = "1"
root = "D://COVID-CTset"
clip_len = 128
batch_size = 1
fold_splitter_test = None
fold_splitter_train = None
    
with open(fold_train_path) as fhandle:
    fold_splitter_train = json.load(fhandle)
    
with open(fold_test_path) as fhandle:
    fold_splitter_test = json.load(fhandle)
    
dataset_test = CTDatasetTestSimple(root=root, 
                      fold_id=fold_id, 
                      fold_splitter=fold_splitter_test,
                      transforms=None,
                      replacer="",
                      prepath="",
                      clip_len=clip_len,
                      split="test"
                      )

dataset_train = CTDatasetTestSimple(root=root, 
                      fold_id=fold_id, 
                      fold_splitter=fold_splitter_train,
                      transforms=None,
                      replacer="",
                      prepath="",
                      clip_len=clip_len,
                      split="train"
                      )

dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True, num_workers=0)
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)

In [ ]:
n_classes = 2

teacher = i3_res50_nl(n_classes)
student = I3Res18(num_classes = n_classes)

teacher_pytorch_total_params = sum(p.numel() for p in teacher.parameters())
student_pytorch_total_params = sum(p.numel() for p in student.parameters())

teacher_layers = len(dict(teacher.named_modules()).keys())
student_layers = len(dict(student.named_modules()).keys())

print('Teacher total parameters: ', end='')
print(teacher_pytorch_total_params)
print('Teacher number of layers: ', end='')
print(teacher_layers)
print("\n------------------------------------------\n")

print('Student total parameters: ', end='')
print(student_pytorch_total_params)
print('Teacher number of layers: ', end='')
print(student_layers)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(teacher.parameters(), lr=0.001, weight_decay=0.0005) 
optimizer3 = torch.optim.ASGD(teacher.parameters(), lr=0.001, lambd=0.0001, alpha=0.75, t0=1000000.0, weight_decay=0)
optimizer2 = torch.optim.SGD
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=10, eps=1e-06)
Adamax = torch.optim.Adamax(teacher.parameters())

In [ ]:
rho = 0.05
learning_rate = 0.01
momentum = 0.9
weight_decay = 0.005
base_optimizer = torch.optim.SGD
optimizer2 = SAM(teacher.parameters(), base_optimizer, rho=rho, lr=learning_rate, momentum=momentum, weight_decay=weight_decay)


In [ ]:
import gc
gc.collect()

optimizerS = torch.optim.Adam(student.parameters(), lr=0.001, weight_decay=0.0005) 
optimizerT = torch.optim.Adam(teacher.parameters(), lr=0.001, weight_decay=0.0005) 

teacher_model = deepcopy(teacher)
student_model = deepcopy(student)
teacher_optimizer = optimizer
student_optimizer = optimizer
loss_function = nn.CrossEntropyLoss()

distiller = VanillaKD(teacher_model, student_model, dataloader_test, dataloader_test, 
                      optimizer2, optimizerS)  

distiller.train_teacher(epochs=10, plot_losses=True, save_model=True)
distiller.train_student(epochs=10, plot_losses=True, save_model=True)
distiller.evaluate(teacher=False)
distiller.get_parameters()

In [ ]:
import gc
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())
print(torch.cuda.memory_summary(device=torch.device("cuda:0"), abbreviated=False))
